## Demo of the Adaptive Ground Point Filtering Library

This notebook gives a tour of the capabilities of `filteradapt`. It is continuously updated while we develop the library. You can execute the code in each cell by pressing `Ctrl+Enter`. To start your own project based on `filteradapt`, you can either edit this notebook or start a new one.

The first thing to do in a Jupyter notebook that uses `filteradapt` is to import the library:

In [ ]:
import adaptivefiltering

The next step you will typically do is to construct a dataset object by reading a LAS/LAZ file:

In [ ]:
#ds = adaptivefiltering.DataSet("/home/gwydion/SSC/ThingstaetteDataset/uls_thingstaette.las")
ds = adaptivefiltering.DataSet("data/500k_NZ20_Westport.laz")

In [ ]:
#ds.show()

In [ ]:
#ds.show_mesh(resolution=2)


In [ ]:
import pdal
import json
import numpy as np
import ipyleaflet
import ipywidgets


The data set can be visualized in the notebook through its `show()` method:

polygon_boundary = ipyleaflet.Polygon(
    locations=boundary_coordinates,
    color="gray",
    opacity=0.9
)

m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldImagery, 
                    center=(coordinates_mean[0], 
                    coordinates_mean[1]), 
                    crs=ipyleaflet.projections.EPSG3857,
                    scroll_wheel_zoom=True,
                    max_zoom=20)
m.add_layer(polygon_boundary)

zoom_slider = ipywidgets.IntSlider(description='Zoom level:', min=0, max=20, value=16)
ipywidgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position='topright')
#m.add_control(widget_control1)



color_picker = ipywidgets.ColorPicker(description='Pick a color:')

draw_control = ipyleaflet.DrawControl(widget = color_picker)

draw_control.polygon = {
    "shapeOptions": {
        "fillColor": color_picker.get_interact_value()
,
        "color": color_picker.get_interact_value()
,
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
def update_draw_control_color( *args, **kwargs):
        m.remove_control(draw_control)
        draw_control.polygon = {"shapeOptions": {
                        "fillColor": color_picker.get_interact_value(),
                        "color": color_picker.get_interact_value(),
                        "fillOpacity": 0.4
                        }}
        m.add_control(draw_control)

widget_control2 = ipyleaflet.WidgetControl(widget=color_picker, position='bottomright')
draw_control.on_draw(update_draw_control_color)

m.add_control(widget_control2)
m.add_control(draw_control)
m

In [ ]:
class interactive_map:

    def __init__(self, dataset):
        """This class manages the interactive map on which one can choose the segmentation. 
           
        :param dataset:
            The dataset from which the map should be displayed. This needs to be in a valid georeferenced format. Eg.: EPSG:4326
        :type dataset: Dataset
        
        """
        self.dataset = dataset
        self.coordinates_mean, self.polygon_boundary = self.get_bounds()

        self.m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldImagery, 
                            center=(self.coordinates_mean[0], 
                            self.coordinates_mean[1]), 
                            crs=ipyleaflet.projections.EPSG3857,
                            scroll_wheel_zoom=True,
                            max_zoom=20)
        self.m.add_layer(self.polygon_boundary)

         #always add polygon draw tool and zoom slider
        self.add_zoom_slider()
        self.add_polygon_control()


        #get a color picker to put on the right side of the map
        self.color_pick = self.get_color_pick()
        #setup the grid with a list of widgets
        self.setup_grid([self.m, self.color_pick])
    
    def get_bounds(self):
        """takes the boundry coordinates of given dataset through the hexbin filter and returns them as a polygon 

        :return:
        :param coordinates_mean: 
            the approximate center of the polygon, this is where the map will be centered
        :type coordinates_mena: ndarray


        :param polygon_boundary: 
            An ipyleaflet Polygon object which can be added to the map to mark the selected area.
        :type polygon_boundary: ipyleaflet Polygon

        """
        hexbin_json = [
            ds.filename,
        {
                "type": "filters.reprojection",
                "out_srs":"EPSG:4326"    
                },
            {
                "type": "filters.hexbin"
            },
            
        ]
        hexbin_pipeline = pdal.Pipeline(json.dumps(hexbin_json))
        hexbin_pipeline.execute()
        hexbin_geojson = json.loads(hexbin_pipeline.metadata)["metadata"]["filters.hexbin"]["boundary_json"]
        #get the previous coordinate representation
        boundary_coordinates = (hexbin_geojson["coordinates"][0])
        coordinates_mean = np.mean(boundary_coordinates,axis=0)
        polygon_boundary = ipyleaflet.Polygon(
            locations=boundary_coordinates,
            color="gray",
            opacity=0.9
        )
        return coordinates_mean, polygon_boundary
      

    def add_zoom_slider(self):
        """Adds the zoom slider to the interactive map. Also sets the default zoom
        """

        self.zoom_slider = ipywidgets.IntSlider(description='Zoom level:', min=0, max=20, value=16)
        ipywidgets.jslink((self.zoom_slider, 'value'), (self.m, 'zoom'))
        self.zoom_control1 = ipyleaflet.WidgetControl(widget=self.zoom_slider, position='topright')
        self.m.add_control(self.zoom_control1)

    def add_polygon_control(self):
        """Adds the polygon draw control.
        """
        self.draw_control = ipyleaflet.DrawControl(layout=ipywidgets.Layout(width='auto', grid_area='main'))
        #deactivate polyline and circlemarker
        self.draw_control.polyline={}
        self.draw_control.circlemarker={}

        self.draw_control.polygon = {
            "shapeOptions": {
                "fillColor": "black"
        ,
                "color": "black"
        ,
                "fillOpacity": 0.1
            },
            "drawError": {
                "color": "#dd253b",
                "message": "Oups!"
            },
            "allowIntersection": False
        }

        self.m.add_control(self.draw_control)

    def get_color_pick(self):
        """Adds the color picker widget"""

        self.color_picker = ipywidgets.ColorPicker(description='Pick a color:',layout=ipywidgets.Layout(width='auto', grid_area='sidebar'))
        self.color_picker.observe(self.update_draw_control_color)
        #self.m.add_control(self.color_control)
        return self.color_picker



    def setup_grid(self, objects):
        """ Setup the grid layout to allow the color bar and more on the right side of the map."""
        self.grid = ipywidgets.GridBox(children=objects,
        layout=ipywidgets.Layout(
            width='100%',
            grid_template_columns='70% 30%',
           # grid_template_rows='90% 10%', 
            grid_template_areas='''
            
            "main sidebar "
            
            ''')
       )

    def draw_handler():
        print("")


    def update_draw_control_color(self,change,  *args, **kwargs):
        """This is the automatic handler for detecting if a new color is selected.

        
        """
        self.m.remove_control(self.draw_control)
        self.draw_control.polygon = {"shapeOptions": {
                        "fillColor": self.color_picker.get_interact_value(),
                        "color": self.color_picker.get_interact_value(),
                        "fillOpacity": 0.1
                        }}
        self.m.add_control(self.draw_control)
        
    def show(self):
        return self.grid

In [ ]:
test_map = interactive_map(ds)
test_map.show()

In [ ]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle
header  = Button(description='Header',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
main    = Button(description='Main',
                 layout=Layout(width='auto', grid_area='main'),
                 style=ButtonStyle(button_color='moccasin'))
sidebar = Button(description='Sidebar',
                 layout=Layout(width='auto', grid_area='sidebar'),
                 style=ButtonStyle(button_color='salmon'))
footer  = Button(description='Footer',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))

GridBox(children=[header, main, sidebar, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "header header header header"
            "main main . sidebar "
            "footer footer footer footer"
            ''')
       )